# 12 - Modelo de Regressão (Prever QUANTO vai vender)

Este notebook treina o segundo estágio do modelo de dois estágios: **regressão**.

## Objetivo:
- Prever a quantidade de vendas **apenas para casos onde vendeu = 1**
- Usar LightGBM Regressor especializado em prever volumes
- Avaliar com métricas de regressão (RMSE, MAE, MAPE)
- Otimizar para casos com vendas positivas

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, 
    r2_score
)
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

print('📊 Iniciando Treinamento do Modelo de Regressão')
print('🎯 Objetivo: Prever QUANTO vai vender (apenas onde vendeu = 1)')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

📊 Iniciando Treinamento do Modelo de Regressão
🎯 Objetivo: Prever QUANTO vai vender (apenas onde vendeu = 1)
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Dados com Features

In [2]:
# Carregar dados com features avançadas
print('📂 Carregando dados com features avançadas...')

train_features = pd.read_parquet('../data/submissao3/train_features.parquet')
validation_features = pd.read_parquet('../data/submissao3/validation_features.parquet')

print(f'🏋️ Dados de treino (completos): {train_features.shape}')
print(f'🔍 Dados de validação (completos): {validation_features.shape}')

# FILTRO CRÍTICO: Manter apenas registros onde vendeu = 1
print('\n✂️ APLICANDO FILTRO CRÍTICO: apenas registros com vendas...')

train_sales = train_features[train_features['vendeu'] == 1].copy()
validation_sales = validation_features[validation_features['vendeu'] == 1].copy()

print(f'🏋️ Dados de treino (apenas vendas): {train_sales.shape}')
print(f'🔍 Dados de validação (apenas vendas): {validation_sales.shape}')

# Verificar distribuição do target de regressão
print(f'\n📊 Estatísticas do target "quantidade" no treino:')
print(f'   • Média: {train_sales["quantidade"].mean():.2f}')
print(f'   • Mediana: {train_sales["quantidade"].median():.2f}')
print(f'   • Desvio padrão: {train_sales["quantidade"].std():.2f}')
print(f'   • Min: {train_sales["quantidade"].min():.0f}')
print(f'   • Max: {train_sales["quantidade"].max():.0f}')

print(f'\n📊 Estatísticas do target "quantidade" na validação:')
print(f'   • Média: {validation_sales["quantidade"].mean():.2f}')
print(f'   • Mediana: {validation_sales["quantidade"].median():.2f}')
print(f'   • Desvio padrão: {validation_sales["quantidade"].std():.2f}')
print(f'   • Min: {validation_sales["quantidade"].min():.0f}')
print(f'   • Max: {validation_sales["quantidade"].max():.0f}')

print('✅ Dados filtrados para regressão')

📂 Carregando dados com features avançadas...
🏋️ Dados de treino (completos): (50126880, 40)
🔍 Dados de validação (completos): (5221550, 40)

✂️ APLICANDO FILTRO CRÍTICO: apenas registros com vendas...
🏋️ Dados de treino (apenas vendas): (5561122, 40)
🔍 Dados de validação (apenas vendas): (572247, 40)

📊 Estatísticas do target "quantidade" no treino:
   • Média: 9.08
   • Mediana: 2.00
   • Desvio padrão: 87.26
   • Min: 0
   • Max: 94230

📊 Estatísticas do target "quantidade" na validação:
   • Média: 5.26
   • Mediana: 2.00
   • Desvio padrão: 15.81
   • Min: 0
   • Max: 2472
✅ Dados filtrados para regressão


## 2. Preparação dos Dados para Regressão

In [ ]:
# Preparar features para regressão - VERSÃO OTIMIZADA
print('🔧 Preparando features para regressão - VERSÃO OTIMIZADA...')

# Carregar lista de features seguras (sem leakage) da classificação
with open('../data/submissao3/classification_features.pkl', 'rb') as f:
    features_classificacao = pickle.load(f)

# Para REGRESSÃO, podemos usar algumas features "atuais" que foram removidas da classificação
# Porque quando fazemos regressão, já sabemos que vendeu=1, então não há leakage
features_extras_para_regressao = []

# Verificar se features "atuais" existem nos dados
todas_features = list(train_sales.columns)
features_atuais_disponiveis = [f for f in todas_features if 'atual' in f]

print(f'📊 Features da classificação (sem leakage): {len(features_classificacao)}')
print(f'📊 Features atuais disponíveis para regressão: {len(features_atuais_disponiveis)}')

if features_atuais_disponiveis:
    print('\n✅ FEATURES EXTRAS PARA REGRESSÃO (com contexto atual):')
    for feat in features_atuais_disponiveis:
        print(f'   • {feat}')
    features_extras_para_regressao = features_atuais_disponiveis
else:
    print('\n📝 Nenhuma feature "atual" encontrada - usando mesmo conjunto da classificação')

# Combinar features: classificação (seguras) + extras para regressão
features_modelo_regressao = features_classificacao + features_extras_para_regressao

print(f'\n📊 Total de features para regressão: {len(features_modelo_regressao)}')
print(f'   📊 Features básicas (da classificação): {len(features_classificacao)}')
print(f'   📊 Features extras (contexto atual): {len(features_extras_para_regressao)}')

# Preparar datasets de regressão
X_train_reg = train_sales[features_modelo_regressao].copy()
y_train_reg = train_sales['quantidade'].copy()

X_val_reg = validation_sales[features_modelo_regressao].copy()
y_val_reg = validation_sales['quantidade'].copy()

print(f'\n📊 Datasets de regressão preparados:')
print(f'   🏋️ X_train_reg: {X_train_reg.shape}, y_train_reg: {y_train_reg.shape}')
print(f'   🔍 X_val_reg: {X_val_reg.shape}, y_val_reg: {y_val_reg.shape}')

# Verificar se há NAs
nas_train = X_train_reg.isnull().sum().sum()
nas_val = X_val_reg.isnull().sum().sum()
print(f'   🧹 NAs no treino: {nas_train}, NAs na validação: {nas_val}')

if nas_train > 0 or nas_val > 0:
    print('   ⚠️ Preenchendo NAs com 0...')
    X_train_reg = X_train_reg.fillna(0)
    X_val_reg = X_val_reg.fillna(0)

# Verificar outliers extremos no target
q99 = y_train_reg.quantile(0.99)
outliers = (y_train_reg > q99).sum()
print(f'\n📊 Análise de outliers:')
print(f'   • P99 da quantidade: {q99:.0f}')
print(f'   • Outliers acima P99: {outliers:,} ({outliers/len(y_train_reg)*100:.2f}%)')

# Para regressão, manter outliers é importante para capturar vendas grandes
print('   💡 Mantendo outliers para preservar informação real de vendas grandes')

print('✅ Dados preparados para regressão com features otimizadas')

## 3. Treinamento do Modelo LightGBM Regressor

In [5]:
# Configurar e treinar LightGBM Regressor
print('🚀 Treinando LightGBM Regressor...')

# Parâmetros otimizados para regressão
lgbm_reg_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 80,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'random_state': 42,
    'n_jobs': -1
}

# Criar modelo
lgbm_regressor = lgb.LGBMRegressor(**lgbm_reg_params)

# Treinar modelo com callbacks para early stopping
print('   📚 Iniciando treinamento...')
lgbm_regressor.fit(
    X_train_reg, y_train_reg,
    eval_set=[(X_val_reg, y_val_reg)],
    eval_metric=['rmse', 'mae'],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

print(f'✅ Modelo treinado! Melhor iteração: {lgbm_regressor.best_iteration_}')
print(f'📊 Score de validação: {lgbm_regressor.best_score_}')

🚀 Treinando LightGBM Regressor...
   📚 Iniciando treinamento...
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 0.603397	valid_0's rmse: 5.12892
Did not meet early stopping. Best iteration is:
[100]	valid_0's l1: 0.603397	valid_0's rmse: 5.12892
✅ Modelo treinado! Melhor iteração: 100
📊 Score de validação: defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('l1', np.float64(0.6033969145163481)), ('rmse', np.float64(5.128924904561696))])})


## 4. Avaliação do Modelo

In [ ]:
# Fazer previsões
print('🔍 Avaliando modelo de regressão...')

# Previsões
y_pred_train_reg = lgbm_regressor.predict(X_train_reg)
y_pred_val_reg = lgbm_regressor.predict(X_val_reg)

# Garantir que previsões não sejam negativas
y_pred_train_reg = np.maximum(y_pred_train_reg, 0)
y_pred_val_reg = np.maximum(y_pred_val_reg, 0)

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1))) * 100

# Métricas de treino
print('\n📊 MÉTRICAS DE TREINO:')
rmse_train = np.sqrt(mean_squared_error(y_train_reg, y_pred_train_reg))
mae_train = mean_absolute_error(y_train_reg, y_pred_train_reg)
r2_train = r2_score(y_train_reg, y_pred_train_reg)
mape_train = mean_absolute_percentage_error(y_train_reg, y_pred_train_reg)

print(f'   📊 RMSE: {rmse_train:.4f}')
print(f'   📊 MAE: {mae_train:.4f}')
print(f'   📊 R²: {r2_train:.4f}')
print(f'   📊 MAPE: {mape_train:.2f}%')

# Métricas de validação
print('\n📊 MÉTRICAS DE VALIDAÇÃO:')
rmse_val = np.sqrt(mean_squared_error(y_val_reg, y_pred_val_reg))
mae_val = mean_absolute_error(y_val_reg, y_pred_val_reg)
r2_val = r2_score(y_val_reg, y_pred_val_reg)
mape_val = mean_absolute_percentage_error(y_val_reg, y_pred_val_reg)

print(f'   📊 RMSE: {rmse_val:.4f}')
print(f'   📊 MAE: {mae_val:.4f}')
print(f'   📊 R²: {r2_val:.4f}')
print(f'   📊 MAPE: {mape_val:.2f}%')

# Análise de resíduos
print('\n📊 ANÁLISE DE RESÍDUOS (Validação):')
residuos = y_val_reg - y_pred_val_reg
print(f'   📊 Média dos resíduos: {residuos.mean():.4f}')
print(f'   📊 Desvio padrão dos resíduos: {residuos.std():.4f}')
print(f'   📊 P25 dos resíduos: {residuos.quantile(0.25):.4f}')
print(f'   📊 P75 dos resíduos: {residuos.quantile(0.75):.4f}')

# Análise por faixas de quantidade
print('\n📊 PERFORMANCE POR FAIXA DE QUANTIDADE:')
faixas = [(0, 5), (5, 10), (10, 20), (20, 50), (50, 1000)]
for faixa_min, faixa_max in faixas:
    mask = (y_val_reg >= faixa_min) & (y_val_reg < faixa_max)
    if mask.sum() > 0:
        mae_faixa = mean_absolute_error(y_val_reg[mask], y_pred_val_reg[mask])
        count_faixa = mask.sum()
        print(f'   Faixa [{faixa_min:3d}, {faixa_max:3d}): MAE={mae_faixa:6.2f}, Count={count_faixa:6,}')

print('✅ Avaliação do modelo de regressão concluída')

## 5. Análise de Importância das Features (Regressão)

In [ ]:
# Analisar importância das features para regressão
print('📊 Analisando importância das features para regressão...')

# Obter importâncias
feature_importance_reg = lgbm_regressor.feature_importances_
feature_names_reg = X_train_reg.columns

# Criar DataFrame de importâncias
importance_reg_df = pd.DataFrame({
    'feature': feature_names_reg,
    'importance': feature_importance_reg
}).sort_values('importance', ascending=False)

# Top 20 features mais importantes para regressão
print('\n🏆 TOP 20 FEATURES MAIS IMPORTANTES (REGRESSÃO):')
for i, (_, row) in enumerate(importance_reg_df.head(20).iterrows()):
    print(f'   {i+1:2d}. {row["feature"]:30s} - {row["importance"]:6.0f}')

# Comparar com importâncias da classificação
print('\n📊 COMPARANDO COM CLASSIFICAÇÃO:')
classification_importance = pd.read_csv('../data/submissao3/classification_feature_importance.csv')

# Top 10 features de cada modelo
top_classification = classification_importance.head(10)['feature'].tolist()
top_regression = importance_reg_df.head(10)['feature'].tolist()

features_em_comum = set(top_classification).intersection(set(top_regression))
print(f'   📊 Features em comum no Top 10: {len(features_em_comum)}/10')
print(f'   📊 Features comuns: {list(features_em_comum)}')

# Análise por categoria de feature para regressão
print('\n📊 IMPORTÂNCIA POR CATEGORIA (REGRESSÃO):')
categorias_feature = {
    'Lag': [f for f in feature_names_reg if 'lag_' in f],
    'Rolling': [f for f in feature_names_reg if any(x in f for x in ['media_4w', 'std_4w', 'max_4w'])],
    'Preço': [f for f in feature_names_reg if 'preco' in f],
    'Calendário': [f for f in feature_names_reg if any(x in f for x in ['mes', 'dia', 'inicio', 'fim'])],
    'Tendência': [f for f in feature_names_reg if any(x in f for x in ['momentum', 'aceleracao'])],
    'Hierarquia': [f for f in feature_names_reg if any(x in f for x in ['media_vendas', 'share'])],
    'Hash': [f for f in feature_names_reg if 'hash' in f]
}

for categoria, features in categorias_feature.items():
    if features:
        importancia_categoria = importance_reg_df[importance_reg_df['feature'].isin(features)]['importance'].sum()
        print(f'   {categoria:12s}: {importancia_categoria:8.0f} ({len(features)} features)')

print('✅ Análise de importância para regressão concluída')

## 6. Salvamento do Modelo de Regressão

In [ ]:
# Salvar modelo de regressão OTIMIZADO
print('💾 Salvando modelo de regressão OTIMIZADO...')

# Salvar modelo
with open('../data/submissao3/lgbm_regressor.pkl', 'wb') as f:
    pickle.dump(lgbm_regressor, f)

# IMPORTANTE: Salvar lista de features da regressão (pode ser diferente da classificação)
with open('../data/submissao3/regression_features.pkl', 'wb') as f:
    pickle.dump(features_modelo_regressao, f)

# Salvar importâncias da regressão
importance_reg_df.to_csv('../data/submissao3/regression_feature_importance.csv', index=False)

# Salvar metadados do modelo de regressão OTIMIZADO
metadados_regressor = {
    'data_criacao': pd.Timestamp.now(),
    'modelo': 'LightGBM Regressor OTIMIZADO',
    'objetivo': 'Regressão: prever quantidade de vendas (apenas onde vendeu = 1)',
    'parametros': lgbm_reg_params,
    'melhor_iteracao': lgbm_regressor.best_iteration_,
    'melhor_score': lgbm_regressor.best_score_,
    'total_features': len(features_modelo_regressao),
    'features_usadas': features_modelo_regressao,
    'features_da_classificacao': len(features_classificacao),
    'features_extras_regressao': len(features_extras_para_regressao),
    'strategy': {
        'base_features': 'Mesmas da classificação (sem leakage)',
        'extra_features': 'Features com contexto atual (permitidas na regressão)',
        'reasoning': 'Regressão pode usar contexto atual pois já sabe que vendeu=1'
    },
    'metricas_validacao': {
        'rmse': rmse_val,
        'mae': mae_val,
        'r2': r2_val,
        'mape': mape_val
    },
    'shape_treino': X_train_reg.shape,
    'shape_validacao': X_val_reg.shape,
    'target_stats_treino': {
        'mean': float(y_train_reg.mean()),
        'median': float(y_train_reg.median()),
        'std': float(y_train_reg.std()),
        'min': float(y_train_reg.min()),
        'max': float(y_train_reg.max())
    },
    'target_stats_validacao': {
        'mean': float(y_val_reg.mean()),
        'median': float(y_val_reg.median()),
        'std': float(y_val_reg.std()),
        'min': float(y_val_reg.min()),
        'max': float(y_val_reg.max())
    },
    'observacoes': [
        'Modelo treinado APENAS em registros com vendeu=1',
        'Pode usar features "atuais" pois não há leakage na regressão',
        'Previsões são clampadas para >= 0',
        'Otimizado para casos com vendas positivas',
        'Preserva outliers para capturar vendas grandes',
        'Deve ser usado em conjunto com classificador'
    ]
}

with open('../data/submissao3/lgbm_regressor_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_regressor, f)

print('✅ Arquivos salvos:')
print('   • data/submissao3/lgbm_regressor.pkl')
print('   • data/submissao3/regression_features.pkl (SEPARADO da classificação)')
print('   • data/submissao3/regression_feature_importance.csv')
print('   • data/submissao3/lgbm_regressor_metadata.pkl')

print('\n🎉 MODELO DE REGRESSÃO OTIMIZADO TREINADO E SALVO!')
print('=' * 70)
print('🎯 Resumo do Modelo de Regressão OTIMIZADO:')
print(f'   📊 RMSE na validação: {rmse_val:.4f}')
print(f'   📊 MAE na validação: {mae_val:.4f}')
print(f'   📊 R² na validação: {r2_val:.4f}')
print(f'   📊 MAPE na validação: {mape_val:.2f}%')
print(f'   📊 Features totais: {len(features_modelo_regressao)}')
print(f'     • Features base (sem leakage): {len(features_classificacao)}')
print(f'     • Features extras (contexto atual): {len(features_extras_para_regressao)}')
print(f'   📊 Registros de treino: {len(X_train_reg):,} (apenas com vendas)')
print(f'   📊 Registros de validação: {len(X_val_reg):,} (apenas com vendas)')

print('\n💡 IMPORTANTE:')
print('   🔄 Classificação e Regressão usam features DIFERENTES')
print('   📊 Classificação: features seguras (sem leakage)')  
print('   📊 Regressão: features seguras + contexto atual')
print('   🎯 Isso é CORRETO e esperado!')

print('\n💡 Próximo passo:')
print('   🔄 Atualizar pipeline final (Notebook 13) para usar features corretas')
print('   📊 Aplicar modelos nas 5 semanas de Janeiro 2023')

print('\n🚀 Segundo estágio OTIMIZADO concluído!')
print('🎯 Ambos os modelos estão prontos e corrigidos!')